In [2]:
from google.colab import files
uploaded = files.upload()

Saving music_genres_dataset.csv to music_genres_dataset.csv


In [3]:
import pandas as pd 
df = pd.read_csv("music_genres_dataset.csv")

df = df.dropna()
df.isnull().sum()

Track               0
Artist              0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
genre               0
dtype: int64

In [4]:
rvar_list = ["Track", "Artist"]
df_sample1 = df.drop(columns = rvar_list)

In [6]:
cvar_list = ["genre", "key", "mode"]
nvar_list = ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms"]

df_sample2 = df_sample1.copy()
def float_transfer(d:"DataFrame", object_list:list):
    for i in object_list:
        d[i] = pd.to_numeric(d[i], errors = "coerce")
    return d
df_sample2 = float_transfer(df_sample1, nvar_list)               
df_sample2[cvar_list] = df_sample1[cvar_list].astype("category") 

In [23]:
df_sample3 = df_sample2.copy()
df_sample3[nvar_list] = (df_sample2[nvar_list] - df_sample2[nvar_list].mean())/df_sample2[nvar_list].std()

df_sample3.isnull().sum()
df_sample4 = df_sample3.dropna()

def irr_check(sample_df: "DataFrame", irr_list: list):
    l = []
    for i in irr_list:
        l.append(sample_df[i].unique())
    return l

df_sample5 = df_sample4.copy() 
df_sample5 = pd.get_dummies(df_sample4, prefix_sep = "_")

error_dummies = ["genre_genre", "key_key", "mode_mode"]
df_sample5 = df_sample5.drop(columns = error_dummies)

rdummies = ["genre_classical", "genre_pop", "genre_Jazz HipHop", "genre_edm", "genre_indie folk"]
df_sample6 = df_sample5.copy() 
df_sample6 = df_sample5.drop(columns = rdummies)

In [24]:
from sklearn.model_selection import train_test_split
df4partition = df_sample6
testpart_size = 0.2
df_nontestData, df_testData = train_test_split(df4partition, test_size=testpart_size, random_state=1)

In [25]:
from sklearn.neighbors import KNeighborsClassifier
DV = "genre_k-pop"
y = df_nontestData[DV]
x = df_nontestData.drop(columns=[DV])

kfolds = 5
max_k = 200
param_grid = {'n_neighbors': list(range(1, max_k+1))}
from sklearn.model_selection import GridSearchCV
gridsearch = GridSearchCV(KNeighborsClassifier(metric = 'euclidean'), param_grid, scoring = 'roc_auc', cv = kfolds, n_jobs=-1)
gridsearch.fit(x,y)
clf_bestKNN = gridsearch.best_estimator_ 

In [26]:
print(clf_bestKNN.n_neighbors)

y_test_actual = df_testData[DV]
x_test = df_testData.drop(columns=[DV])
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test_actual, clf_bestKNN.predict_proba(x_test)[:, 1]))

50
0.9198212065597062
